In [ ]:
! pip install -U kaleido && pip install git+https://github.com/jkminder/pycolors.git && pip install -e .../dictionary_learning

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from transformers import AutoModelForCausalLM, AutoTokenizer
from dictionary_learning.cache import ActivationCache
from dictionary_learning.dictionary import CrossCoder
from datasets import load_from_disk, load_dataset
from loguru import logger
import torch as th
from nnsight import LanguageModel
from pathlib import Path
from pycolors import TailwindColorPalette
import os
from tools.feature_analysis import plot_feature_freq, plot_feature_diff
from pathlib import Path
# from dlabutils import model_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
cc_name = "l13-mu4.1e-02-lr1e-04"
stats_path = Path("../stats/")
save_dir = Path(f"../plots/") / model_name
base_dir = Path(f"../results/eval_crosscoder/{cc_name}_ae_final/data")

os.makedirs(save_dir, exist_ok=True)

stats_fineweb = th.load(stats_path / model_name / "fineweb.pt", map_location="cpu")
stats_lmsys = th.load(stats_path / model_name / "lmsys.pt", map_location="cpu")

In [ ]:
for rescaled in [True, False]:
    for split in ["joint", "instruction", "base"]:
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="base",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="instruction",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="shared",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name=None,
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )

In [ ]:
for rescaled in [True, False]:
    for group in ["base", "instruction", "shared", None]:
        plot_feature_diff(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name=group,
            rescaled=rescaled,
            indices_path=base_dir,
            save=True,
        )

In [11]:
stats_lmsys

CombinedFeatureStatistics(rescaled=FeatureStatistics(base=FeatureStatistic(avg_activation=tensor([2.2304e-03, 1.0332e-03, 7.5188e-06,  ..., 9.1480e-04, 4.0949e-04,
        1.1720e-04]), non_zero_counts=tensor([10947,  6632,   107,  ...,  1240, 12559,   622]), total_tokens=tensor(11930591), is_normalized=True), instruction=FeatureStatistic(avg_activation=tensor([0.0147, 0.0014, 0.0002,  ..., 0.0016, 0.0015, 0.0003]), non_zero_counts=tensor([133964,   6756,   2800,  ...,   2771,   9807,   1869]), total_tokens=tensor(11930591), is_normalized=True), joint=FeatureStatistic(avg_activation=tensor([2.0735e-03, 2.1034e-03, 1.0883e-05,  ..., 4.4947e-03, 2.0572e-03,
        4.9375e-04]), non_zero_counts=tensor([8070, 1591,   46,  ..., 1170, 3028,  374]), total_tokens=tensor(11930591), is_normalized=True), abs_activation_diff=tensor([-0.0125, -0.0004, -0.0002,  ..., -0.0007, -0.0011, -0.0001]), rel_activation_diff=tensor([0.0717, 0.4868, 0.0349,  ..., 0.2005, 0.5493, 0.2123]), either_non_zero_coun

In [25]:
# Get Fossil Features
# Load instruction only indices
from tools.feature_utils import indices_to_mask, mask_to_indices
indices = th.load(base_dir / "only_it_decoder_feature_indices.pt", map_location="cpu")
mask = indices_to_mask(indices, stats_lmsys.rescaled.base.avg_activation.shape[0])

/tmp/ipykernel_1912165/3879260724.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  indices = th.load(base_dir / "only_it_decoder_feature_indices.pt", map_location="cpu")


In [26]:
lmsys_it_fossil_mask = mask & (stats_lmsys.rescaled.rel_activation_diff > 0.95)
lmsys_it_fossil_indices = mask_to_indices(lmsys_it_fossil_mask)
len(lmsys_it_fossil_indices)

45

In [27]:
th.save(lmsys_it_fossil_indices, base_dir / "lmsys_it_fossil_indices.pt")

In [28]:
base_dir / "lmsys_it_fossil_indices.pt"

PosixPath('/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data/lmsys_it_fossil_indices.pt')

In [31]:
for fossil in lmsys_it_fossil_indices:
    assert fossil in indices

In [ ]:
stats_lmsys

In [ ]:
model_name = "l13-mu4.1e-02-lr1e-04"
stats_path = Path(f"/dlabscratch1/jminder/repositories/representation-structure-comparison/stats/")
save_dir = Path(f"/dlabscratch1/jminder/repositories/representation-structure-comparison/plots/") / model_name
base_dir = Path(f"/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data")

os.makedirs(save_dir, exist_ok=True)

stats_fineweb = th.load(stats_path / model_name / "fineweb.pt").to("cpu")
stats_lmsys = th.load(stats_path / model_name / "lmsys.pt").to("cpu")

In [ ]:
lmsys_it_fossil_mask = mask & (stats_lmsys.rescaled.rel_activation_diff > 0.95)
lmsys_it_fossil_indices = mask_to_indices(lmsys_it_fossil_mask)
len(lmsys_it_fossil_indices)

In [27]:
th.save(lmsys_it_fossil_indices, base_dir / "lmsys_it_fossil_indices.pt")

In [ ]:
base_dir / "lmsys_it_fossil_indices.pt"

In [31]:
for fossil in lmsys_it_fossil_indices:
    assert fossil in indices

In [ ]:
stats_lmsys

In [ ]:
# Get Fossil Features
# Load instruction only indices
from tools.feature_utils import indices_to_mask, mask_to_indices
indices = th.load(base_dir / "only_it_decoder_feature_indices.pt", map_location="cpu")
mask = indices_to_mask(indices, stats_lmsys.rescaled.base.avg_activation.shape[0])

In [ ]:
lmsys_it_fossil_mask = mask & (stats_lmsys.rescaled.rel_activation_diff > 0.95)
lmsys_it_fossil_indices = mask_to_indices(lmsys_it_fossil_mask)
len(lmsys_it_fossil_indices)

In [27]:
th.save(lmsys_it_fossil_indices, base_dir / "lmsys_it_fossil_indices.pt")

In [ ]:
base_dir / "lmsys_it_fossil_indices.pt"

In [31]:
for fossil in lmsys_it_fossil_indices:
    assert fossil in indices